In [1]:

import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt

# from keras import backend as K
# from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
# from keras.models import Model
# from keras.layers.normalization import BatchNormalization
# from keras.layers.pooling import MaxPooling2D, AveragePooling2D
# from keras.layers.core import Lambda, Flatten, Dense

In [2]:
# Following the Paper from inception module

# The first three layers are the simple conv layer followed 
    # conv -> Maxpool -> BN -> conv -> conv -> BN -> Maxpool
# Now the inception bloacks starts
    # Block = Layer1 [1x1, 1x1, Maxpool], Layer2 [1x1, 3x3, 5x5, 1x1]
    # 2* [Block -> concat] -> Maxpool     
    # 5 * [Block -> concat] -> Maxpool
    # 2* [Block -> concat] -> Maxpool 
    
    
# The Facenet model have a slightly different architecture when compared to inception modules
convShape = {
    'conv1': [64, 3, 7, 7],
    'conv2': [64, 64, 1, 1],
    'conv3': [192, 64, 3, 3],
# Block 1
    # Chain 1
    'inception_3a_1x1_conv': [64, 192, 1, 1],
    # Chain 2
    'inception_3a_3x3_conv1': [96, 192, 1, 1],
    'inception_3a_3x3_conv2': [128, 96, 3, 3],
    # Chain 3
    'inception_3a_5x5_conv1': [16, 192, 1, 1],
    'inception_3a_5x5_conv2': [32, 16, 5, 5],
    # Chain 4
    'inception_3a_pool_conv': [32, 192, 1, 1],
   
# Block 2
    # Chain 1
    'inception_3b_1x1_conv': [64, 256, 1, 1],
    # Chain 2
    'inception_3b_3x3_conv1': [96, 256, 1, 1],
    'inception_3b_3x3_conv2': [128, 96, 3, 3],
    # Chain 3
    'inception_3b_5x5_conv1': [32, 256, 1, 1],
    'inception_3b_5x5_conv2': [64, 32, 5, 5],
    # Chain 4
    'inception_3b_pool_conv': [64, 256, 1, 1],
   
# Block 3  [Note: No chain 1 and 4]
    # Chain 2
    'inception_3c_3x3_conv1': [128, 320, 1, 1],
    'inception_3c_3x3_conv2': [256, 128, 3, 3],
    # Chain 3 : 
    'inception_3c_5x5_conv1': [32, 320, 1, 1],
    'inception_3c_5x5_conv2': [64, 32, 5, 5],

# Block 4
    # Chain 1
    'inception_4a_1x1_conv': [256, 640, 1, 1],
    # Chain 2
    'inception_4a_3x3_conv1': [96, 640, 1, 1],
    'inception_4a_3x3_conv2': [192, 96, 3, 3],
    # Chain 3
    'inception_4a_5x5_conv1': [32, 640, 1, 1,],
    'inception_4a_5x5_conv2': [64, 32, 5, 5],
    # Chain 4 
    'inception_4a_pool_conv': [128, 640, 1, 1],

# Block 5
    # Chain 2
    'inception_4e_3x3_conv1': [160, 640, 1, 1],
    'inception_4e_3x3_conv2': [256, 160, 3, 3],
    # Chain 3
    'inception_4e_5x5_conv1': [64, 640, 1, 1],
    'inception_4e_5x5_conv2': [128, 64, 5, 5],

# Block 6
    # Chain 1
    'inception_5a_1x1_conv': [256, 1024, 1, 1],
    # Chain 2
    'inception_5a_3x3_conv1': [96, 1024, 1, 1],
    'inception_5a_3x3_conv2': [384, 96, 3, 3],
    # Chain 3
    'inception_5a_pool_conv': [96, 1024, 1, 1],
  
# Bloack 7
    # Chain 1
    'inception_5b_1x1_conv': [256, 736, 1, 1],
    # Chain 2
    'inception_5b_3x3_conv1': [96, 736, 1, 1],
    'inception_5b_3x3_conv2': [384, 96, 3, 3],
    # Chain 3
    'inception_5b_pool_conv': [96, 736, 1, 1],
  
}

In [3]:
picPath = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceRecognition/camera_0.jpg'

from scipy import misc

def readImage(imagePath):
    '''
        The input data is is in the shape of [nh, nw, nc], convert it to [nc, nh, nw]
    '''
    image = misc.imread(picPath)
    
    img = np.around(np.transpose(image, (2,0,1))/255.0, decimals=12) #(2,0,1) = [nc, nh, nw]
    print (image.shape)

## Load Image

In [4]:
picPath = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceRecognition/camera_0.jpg'

from scipy import misc

def readImage(imagePath):
    '''
        The input data is is in the shape of [nh, nw, nc], convert it to [nc, nh, nw]
    '''
    image = misc.imread(picPath)
    
    img = np.around(image/255.0, decimals=12) #(2,0,1) = [nc, nh, nw]
    print (image.shape)
    return img

## Load Weights

In [5]:
from collections import defaultdict
moduleWeightDict = defaultdict(lambda: defaultdict())
parentPath = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceNet-Inception/"
layer_name = ['conv1', 'conv2', 'conv3', 
              'inception_3a_1x1_conv', 
              'inception_3a_3x3_conv1', 
              'inception_3a_3x3_conv2',
              'inception_3a_5x5_conv1', 
              'inception_3a_5x5_conv2', 
              'inception_3a_pool_conv',
              'inception_3b_1x1_conv', 
              'inception_3b_3x3_conv1', 
              'inception_3b_3x3_conv2',
              'inception_3b_5x5_conv1', 
              'inception_3b_5x5_conv2', 
              'inception_3b_pool_conv',
              'inception_3c_3x3_conv1',
              'inception_3c_3x3_conv2',
              'inception_3c_5x5_conv1', 
              'inception_3c_5x5_conv2',
              'inception_4a_3x3_conv1', 
              'inception_4a_3x3_conv2', 
              'inception_4a_5x5_conv1', 
              'inception_4a_5x5_conv2', 
              'inception_4a_pool_conv', 
              'inception_4a_1x1_conv',
              'inception_4e_3x3_conv1',
              'inception_4e_3x3_conv2',
              'inception_4e_5x5_conv1', 
              'inception_4e_5x5_conv2',
              'inception_5a_1x1_conv',
              'inception_5a_3x3_conv1',
              'inception_5a_3x3_conv2',
              'inception_5a_pool_conv',
              'inception_5b_3x3_conv1',
              'inception_5b_3x3_conv2',
              'inception_5b_pool_conv',
              'inception_5b_1x1_conv']#, 'conv1_w']

for name in layer_name:
    conv1_w = np.genfromtxt(parentPath+"/"+name+'_w.csv', delimiter=',', dtype=None)
    conv1_w = np.reshape(conv1_w, convShape[name])
    conv1_w = np.transpose(conv1_w, (2, 3, 1, 0))
    
    conv1_b = np.genfromtxt(parentPath+"/"+name+'_b.csv', delimiter=',', dtype=None)
    moduleWeightDict[name]['w'] = conv1_w
    moduleWeightDict[name]['b'] = conv1_b
    print (name)
    print (conv1_w.shape)
    print (conv1_b.shape)
# FRmodel.get_layer('conv1').set_weights([conv1_w, conv1_b])

conv1
(7, 7, 3, 64)
(64,)
conv2
(1, 1, 64, 64)
(64,)
conv3
(3, 3, 64, 192)
(192,)
inception_3a_1x1_conv
(1, 1, 192, 64)
(64,)
inception_3a_3x3_conv1
(1, 1, 192, 96)
(96,)
inception_3a_3x3_conv2
(3, 3, 96, 128)
(128,)
inception_3a_5x5_conv1
(1, 1, 192, 16)
(16,)
inception_3a_5x5_conv2
(5, 5, 16, 32)
(32,)
inception_3a_pool_conv
(1, 1, 192, 32)
(32,)
inception_3b_1x1_conv
(1, 1, 256, 64)
(64,)
inception_3b_3x3_conv1
(1, 1, 256, 96)
(96,)
inception_3b_3x3_conv2
(3, 3, 96, 128)
(128,)
inception_3b_5x5_conv1
(1, 1, 256, 32)
(32,)
inception_3b_5x5_conv2
(5, 5, 32, 64)
(64,)
inception_3b_pool_conv
(1, 1, 256, 64)
(64,)
inception_3c_3x3_conv1
(1, 1, 320, 128)
(128,)
inception_3c_3x3_conv2
(3, 3, 128, 256)
(256,)
inception_3c_5x5_conv1
(1, 1, 320, 32)
(32,)
inception_3c_5x5_conv2
(5, 5, 32, 64)
(64,)
inception_4a_3x3_conv1
(1, 1, 640, 96)
(96,)
inception_4a_3x3_conv2
(3, 3, 96, 192)
(192,)
inception_4a_5x5_conv1
(1, 1, 640, 32)
(32,)
inception_4a_5x5_conv2
(5, 5, 32, 64)
(64,)
inception_4a_pool

In [6]:
from Model import getModel

def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

img = readImage(picPath)
# img = np.reshape(img, (1,img.shape[0], img.shape[1], img.shape[2]))
img.shape
reset_graph()
tensorDict = getModel(img.shape, params=moduleWeightDict)#(img, conv1_w, conv1_b, s=2, pad='SAME',  scope_name='conv1', isTrainable=False)

(96, 96, 3)
inpTensor  (?, 96, 96, 3)
conv1:  (?, 48, 48, 64)
conv1 Zero-Padding + MAXPOOL  (?, 50, 50, 64)
conv1 Zero-Padding + MAXPOOL  (?, 24, 24, 64)
conv2:  (?, 24, 24, 64)
conv2 Zero-Padding + MAXPOOL  (?, 26, 26, 64)
conv3:  (?, 24, 24, 192)
conv3 Zero-Padding + MAXPOOL  (?, 26, 26, 192)
conv3 Zero-Padding + MAXPOOL  (?, 12, 12, 192)
Inside Inception module1:  (?, 12, 12, 192)
inception_3x3inception_3x3inception_3x3inception_3x3inception_3x3
dcev442232323232322323  (?, 12, 12, 192)
FILTERFILTERFILTERFILTER (1, 1, 192, 96)
sdsdfsdfsdfsdfsd  Tensor("Relu:0", shape=(?, 12, 12, 96), dtype=float32)
23123232323232323  Tensor("Pad_1:0", shape=(?, 14, 14, 96), dtype=float32) (3, 3, 96, 128)
76gg6y65yghth66hgf  Tensor("add_1:0", shape=(?, 12, 12, 128), dtype=float32)


/Users/sam/App-Setup/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


Chain 2:  (?, 12, 12, 128)
inception_5x5inception_5x5inception_5x5inception_5x5inception_5x5
dcev442232323232322323  (?, 12, 12, 192)
FILTERFILTERFILTERFILTER (1, 1, 192, 16)
sdsdfsdfsdfsdfsd  Tensor("Relu_2:0", shape=(?, 12, 12, 16), dtype=float32)
23123232323232323  Tensor("Pad_2:0", shape=(?, 16, 16, 16), dtype=float32) (5, 5, 16, 32)
76gg6y65yghth66hgf  Tensor("add_3:0", shape=(?, 12, 12, 32), dtype=float32)
Chain 3:  (?, 12, 12, 32)
76gg6y65yghth66hgf  Tensor("conv3/max_pooling2d/MaxPool:0", shape=(?, 12, 12, 192), dtype=float32)
AveragePooling2D AveragePooling2D AveragePooling2D (?, 5, 5, 192)
Conv2DConv2DConv2DConv2DConv2D  Tensor("Relu_4:0", shape=(?, 5, 5, 32), dtype=float32)
Chain 4:  (?, 12, 12, 32)
Chain 1:  (?, 12, 12, 64)
inception3a:  (?, 12, 12, 256)
Inside Inception module1:  (?, 12, 12, 256)
inception_3x3inception_3x3inception_3x3inception_3x3inception_3x3
dcev442232323232322323  (?, 12, 12, 256)
FILTERFILTERFILTERFILTER (1, 1, 256, 96)
sdsdfsdfsdfsdfsd  Tensor("Relu_

In [6]:
from nn.Inception import getModel

def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

img = readImage(picPath)
# img = np.reshape(img, (1,img.shape[0], img.shape[1], img.shape[2]))
img.shape
reset_graph()
tensorDict = getModel(img.shape, params=moduleWeightDict)#(img, conv1_w, conv1_b, s=2, pad='SAME',  scope_name='conv1', isTrainable=False)

(96, 96, 3)
inpTensor  (?, 96, 96, 3)
conv1:  (?, 48, 48, 64)
conv1 Zero-Padding + MAXPOOL  (?, 50, 50, 64)
conv1 Zero-Padding + MAXPOOL  (?, 24, 24, 64)
conv2:  (?, 24, 24, 64)
conv2 Zero-Padding + MAXPOOL  (?, 26, 26, 64)
conv3:  (?, 24, 24, 192)
conv3 Zero-Padding + MAXPOOL  (?, 26, 26, 192)
conv3 Zero-Padding + MAXPOOL  (?, 12, 12, 192)
Inside Inception module1:  (?, 12, 12, 192)


/Users/sam/App-Setup/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


Chain 2:  (?, 12, 12, 128)
Chain 3:  (?, 12, 12, 32)
Chain 4:  (?, 12, 12, 32)
Chain 1:  (?, 12, 12, 64)
inception3a:  (?, 12, 12, 256)
Inside Inception module1:  (?, 12, 12, 256)
Chain 2:  (?, 12, 12, 128)
Chain 3:  (?, 12, 12, 64)
Chain 4:  (?, 12, 12, 64)
Chain 1:  (?, 12, 12, 64)
inception3b:  (?, 12, 12, 320)


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     print (img.shape)
    img1 = np.reshape(img, (1,img.shape[0], img.shape[1], img.shape[2]))
    print (img1.shape)
    x = sess.run([tensorDict['output']], feed_dict={tensorDict['inpTensor']:img1})
    print (x[0].shape)